# 네이버 OpenAPI 연동 (검색어 트렌드)

## #01. 준비과정
### [1] 패키지 참조

In [ ]:
import requests
import json
from pandas import DataFrame
import datetime as dt
import os
from dotenv import load_dotenv



### [2] 접속할 데이터의 URL

In [ ]:
url = "https://openapi.naver.com/v1/datalab/search"

### [3] 요청 변수

#### 네이버에서 발급받은 인증키

In [ ]:
load_dotenv(dotenv_path = '../../../dot_env/.env')
clientId = os.environ.get("naverId")
clientSecret = os.environ.get("naverPw")

#### 검색 데이터 구성

In [ ]:
now = dt.datetime.now()
today = now.strftime("%Y-%m-%d")
today


In [ ]:
td = dt.timedelta(days=365)
td


In [ ]:
startDateDelta = now - td
startDate = startDateDelta.strftime("%Y-%m-%d")
startDate

In [ ]:
params  = {
    "startDate" : startDate,
    "endDate" : today,
    "timeUnit" : "date", # date: 일간, week: 주간, month: 월간
    "keywordGroups": [
        {
            "groupName": "Benz",  # 검색주제
            "keywords": ["벤츠", "e클래스", "s클래스", "c클래스", "벤츠 cls"]   # 주제와 관련된 검색어
        },
        {
            "groupName": "BMW",  # 검색주제
            "keywords": ["BMW", "5시리즈", "x5", "x7", "x3"]   # 주제와 관련된 검색어
        },
        {
            "groupName": "AUDI",  # 검색주제
            "keywords": ["아우디", "a6", "e트론", "a7", "q5"]   # 주제와 관련된 검색어
        }
    ]
}


### [4] 세션 생성

In [ ]:
session = requests.Session()

session.headers.update({
    "Referer": "",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "X-Naver-Client-Id": clientId,
    "X-Naver-Client-Secret": clientSecret
})


## #02. 데이터 요청하기

### [1] URL을 통한 데이터 요청

In [ ]:
try:
    r = session.post(url, data=json.dumps(params)) # json.dumps : 딕셔너리를 json으로 만들어주는 메서드
    
    if r.status_code != 200:
        msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
        raise Exception(msg)
except Exception as e:
    print("접속에 실패했습니다.")
    print(e)


### [2] 응답 결과 확인

In [ ]:
r.encoding = "utf-8"
mydict = json.loads(r.text)

print("검색 결과 수 :", len(mydict['results']))
mydict

## 03. 응답 데이터 활용

### [1] 데이터 재구성

```python
[
    {"날짜": "2022-12-15", "Benz": 100, "BMW": 100, "AUDI": 100},
    {"날짜": "2022-12-15", "Benz": 100, "BMW": 100, "AUDI": 100},
    {"날짜": "2022-12-15", "Benz": 100, "BMW": 100, "AUDI": 100},
    {"날짜": "2022-12-15", "Benz": 100, "BMW": 100, "AUDI": 100}
]
```

#### 검색 결과의 `title`추출하여 데이터 구조 정의

In [ ]:
struct = {'date': None}

for item in mydict['results']:
    struct[item['title']] = None
    
struct

#### 응답 결과를 탐색하면서 주어진 구조에 데이터 채워 넣기

In [ ]:
mydata = []

for i,v in enumerate(mydict['results']):
    for j,w in enumerate(v['data']):
        if i ==0 :
            temp = struct.copy()
            temp['date'] = w['period']
            mydata.append(temp)
        mydata[j][f'{v['title']}'] = w['ratio']

        
mydata